# Bangalore House Price Prediction




1. Pratik Thapa
2. Pankaj Kumar Barman
3. Manish Gupta








# Importing Required Modules


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import joblib
!pip install fastapi nest-asyncio pyngrok uvicorn
from fastapi import FastAPI
import nest_asyncio
from pyngrok import ngrok
import uvicorn
import pickle
from pydantic import BaseModel



     |████████████████████████████████| 51kB 2.8MB/s 
     |████████████████████████████████| 747kB 8.2MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
     |████████████████████████████████| 61kB 9.4MB/s 
     |████████████████████████████████| 10.1MB 27.5MB/s 
     |████████████████████████████████| 61kB 7.5MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.0.5-cp37-none-any.whl size=19262 sha256=6250d797c29139fc9b435e983a015fcd092934c2087b9e7d9475dc515157a991
  Stored in directory: /root/.cache/pip/wheels/0c/13/64/5ebbcc22eaf53fdf5766b397c1fb17c83f5775fdccf0ea1b88
Successfully built pyngrok


# Loading CSV File and Formatting Data

In [3]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [4]:
file = r"https://drive.google.com/uc?export=download&id=1P49POlAk27uRzWKXoR2WaEfb1lyyfiRJ"

table = pd.read_csv(file)
table.shape

(7120, 109)

In [5]:
table.shape

(7120, 109)

In [6]:
table.head()

,Unnamed: 0,bath,balcony,price,total_sqft_int,bhk,price_per_sqft,area_typeSuper built-up Area,area_typeBuilt-up Area,area_typePlot Area,availability_Ready To Move,location_Whitefield,location_Sarjapur Road,location_Electronic City,location_Raja Rajeshwari Nagar,location_Marathahalli,location_Haralur Road,location_Hennur Road,location_Bannerghatta Road,location_Uttarahalli,location_Thanisandra,location_Electronic City Phase II,location_Hebbal,location_Yelahanka,location_7th Phase JP Nagar,location_Kanakpura Road,location_KR Puram,location_Sarjapur,location_Rajaji Nagar,location_Bellandur,location_Kasavanhalli,location_Begur Road,location_Kothanur,location_Banashankari,location_Hormavu,location_Harlur,location_Akshaya Nagar,location_Jakkur,location_Electronics City Phase 1,location_Varthur,location_HSR Layout,location_Chandapura,location_Ramamurthy Nagar,location_Hennur,location_Kundalahalli,location_Ramagondanahalli,location_Kaggadasapura,location_Koramangala,location_Hulimavu,location_Budigere,location_Hoodi,location_Malleshwaram,location_JP Nagar,location_Hegde Nagar,location_Yeshwanthpur,location_8th Phase JP Nagar,location_Gottigere,location_Channasandra,location_Bisuvanahalli,location_Vittasandra,location_Indira Nagar,location_Brookefield,location_Vijayanagar,location_Old Airport Road,location_Sahakara Nagar,location_Hosa Road,location_Kengeri,location_Bommasandra,location_Green Glen Layout,location_Balagere,location_Old Madras Road,location_Kudlu Gate,location_Panathur,location_Rachenahalli,location_Yelahanka New Town,location_Jigani,location_Kadugodi,location_Talaghattapura,location_Thigalarapalya,location_Mysore Road,location_Ambedkar Nagar,location_Frazer Town,location_Dodda Nekkundi,location_Attibele,location_Kanakapura,location_Devanahalli,location_Nagarbhavi,location_TC Palaya,location_Lakshminarayana Pura,location_5th Phase JP Nagar,location_Anekal,location_Ananth Nagar,location_Jalahalli,location_CV Raman Nagar,location_Kengeri Satellite Town,location_Kudlu,location_Bhoganhalli,location_Doddathoguru,location_Subramanyapura,location_Kalena Agrahara,location_Horamavu Agara,location_Hosur Road,location_Vidyaranyapura,location_Hebbal Kempapura,location_BTM 2nd Stage,location_Mahadevpura,location_Horamavu Banaswadi,location_Domlur,location_Tumkur Road
0,0,3.0,2.0,150.0,1672.0,3,8971.291866,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,3.0,3.0,149.0,1750.0,3,8514.285714,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,3.0,2.0,150.0,1750.0,3,8571.428571,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,2.0,2.0,40.0,1250.0,2,3200.000000,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,2.0,2.0,83.0,1200.0,2,6916.666667,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [7]:
table = table.drop(['Unnamed: 0'], axis=1)
table.drop(table.iloc[:, 6:-1], inplace = True, axis = 1)
table = table.drop(['location_Tumkur Road'], axis=1)
table.head()

,bath,balcony,price,total_sqft_int,bhk,price_per_sqft
0,3.0,2.0,150.0,1672.0,3,8971.291866
1,3.0,3.0,149.0,1750.0,3,8514.285714
2,3.0,2.0,150.0,1750.0,3,8571.428571
3,2.0,2.0,40.0,1250.0,2,3200.000000
4,2.0,2.0,83.0,1200.0,2,6916.666667


In [8]:
table.describe()

,bath,balcony,price,total_sqft_int,bhk,price_per_sqft
count,7120.000000,7120.000000,7120.000000,7120.000000,7120.000000,7120.000000
mean,2.390871,1.572759,96.454991,1479.729806,2.465169,5923.806855
std,0.876822,0.770583,116.185034,913.779769,0.841520,2556.650935
min,1.000000,0.000000,10.000000,350.000000,1.000000,1250.000000
25%,2.000000,1.000000,49.230000,1100.000000,2.000000,4416.761042
50%,2.000000,2.000000,69.000000,1255.000000,2.000000,5417.855613
75%,3.000000,2.000000,104.000000,1640.250000,3.000000,6618.285651
max,9.000000,3.000000,2912.000000,30400.000000,9.000000,35000.000000


# Data Splitting

In [9]:
X = table.drop("price", axis=1)
y = table['price']
print('Shape of X = ', X.shape)
print('Shape of y = ', y.shape)

Shape of X =  (7120, 5)
Shape of y =  (7120,)


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 51)
print('Shape of X_train = ', X_train.shape)
print('Shape of y_train = ', y_train.shape)
print('Shape of X_test = ', X_test.shape)
print('Shape of y_test = ', y_test.shape)

Shape of X_train =  (5696, 5)
Shape of y_train =  (5696,)
Shape of X_test =  (1424, 5)
Shape of y_test =  (1424,)


## Data Normalisation

In [11]:
sc = StandardScaler()
sc.fit(X_train)
X_train= sc.transform(X_train)
X_test = sc.transform(X_test)

# Algorithms

## Linear Regression

In [12]:
lr = LinearRegression()

In [13]:
def rmse(y_test, y_pred):
  return np.sqrt(mean_squared_error(y_test, y_pred))

In [14]:
lr.fit(X_train, y_train)
lr_eff = lr.score(X_test, y_test)
lr_rmse = rmse(y_test, lr.predict(X_test))
lr_eff, lr_rmse

(0.7837532911322952, 65.91685277030534)

## XGBoost

In [15]:
import xgboost
xgb_reg = xgboost.XGBRegressor()
xgb_reg.fit(X_train,y_train)
xgb_reg_eff=xgb_reg.score(X_test,y_test)
xgb_reg_rmse = rmse(y_test, xgb_reg.predict(X_test))
xgb_reg_eff, xgb_reg_rmse

[10:32:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(0.8833515313052481, 48.41291483271873)

In [16]:
print(pd.DataFrame([{'Model': 'Linear Regression','Efficiency':lr_eff, "RMSE":lr_rmse},
              {'Model': 'XGBoost','Efficiency':xgb_reg_eff, "RMSE":xgb_reg_rmse}],
             columns=['Model','Efficiency','RMSE']))

               Model  Efficiency       RMSE
0  Linear Regression    0.783753  65.916853
1            XGBoost    0.883352  48.412915


# Predictor

In [17]:
def predictor(model,bath,balcony,total_sqft_int,bhk,price_per_sqft):
  x =np.zeros(len(X.columns))
  x[0]=bath
  x[1]=balcony
  x[2]=total_sqft_int
  x[3]=bhk
  x[4]=price_per_sqft
  x = sc.transform([x])[0]
  return model.predict([x])[0]

# Saving and Loading the Model

In [18]:
joblib.dump(xgb_reg, 'model.pkl')

['model.pkl']

In [19]:
model = joblib.load("model.pkl")

[10:32:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [20]:
predictor(model,bath=1,balcony=0,total_sqft_int=300,bhk=1,price_per_sqft=6000)

34.905094

#Deploying the Model

In [21]:
app = FastAPI()
pickle_in = open('model.pkl',"rb")
reg = pickle.load(pickle_in)

@app.get('/Blog')
def fun():
	return {'name':"BHPP"}

@app.post('/predict')
class Features(BaseModel):
	bath:int
	balcony:int
	TotalSquareFit: int
	bhk: int
	PricePerSqfit: int
def prediction(data:Features):
	data = data.dict()
	bath = data['bath']
	balcony = data['balcony']
	TotalSquareFit = data['TotalSquareFit']
	bhk = data['bhk']
	PricePerSqfit = data['PricePerSqfit']
	prediction = reg.predict([[bath,balcony,TotalSquareFit,bhk,PricePerSqfit]])
	return {
    'Predicted Price ': prediction[0]
    }

[10:32:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [22]:
ngrok.kill()
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)